In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/Colab Notebooks/homework/CIS5810/Final_Project NST/InST

/content/drive/MyDrive/Colab Notebooks/homework/CIS5810/Final_Project NST/InST


In [3]:
# !git clone https://github.com/zyxElsa/InST.git

In [4]:
# ! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
# ! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
# ! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
# import sys
# sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [5]:
# import os
# os.chdir('InST/')

In [6]:
# ls

In [7]:
# !conda env create -f environment.yaml
# !conda activate ldm

In [ ]:
!pip3 install omegaconf
!pip3 install einops
!pip3 install pytorch-lightning==1.6.5
!pip3 install torchmetrics==0.11.4
!pip3 install taming-transformers
!pip3 install clip
!pip3 install kornia
# !pip3 install torchtext==0.10.0
# !pip3 install transformers

In [9]:
!pip3 install test-tube
!pip3 install streamlit
!pip3 install torch-fidelity==0.3.0
!pip3 install pudb==2019.2
!pip3 install imageio==2.14.1
!pip3 install imageio-ffmpeg==0.4.7
!pip3 install cudatoolkit==11.3
!pip3 install albumentations==1.1.0

ERROR: Could not find a version that satisfies the requirement cudatoolkit==11.3 (from versions: none)
ERROR: No matching distribution found for cudatoolkit==11.3


In [10]:
"""make variations of input image"""

import argparse, os, sys, glob
import PIL
import torch
import torch.nn as nn
import numpy as np
from omegaconf import OmegaConf
from PIL import Image
from tqdm import tqdm, trange
from itertools import islice
from einops import rearrange, repeat
from torchvision.utils import make_grid
from torch import autocast
from contextlib import nullcontext
import time
from pytorch_lightning import seed_everything
import pytorch_lightning
sys.path.append(os.path.dirname(sys.path[0]))
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler

from transformers import CLIPProcessor, CLIPModel

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())


def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    model.to(device)
    model.eval()
    return model


def load_img(path):
    image = Image.open(path).convert("RGB")
    w, h = image.size
    print(f"loaded input image of size ({w}, {h}) from {path}")
    w, h = map(lambda x: x - x % 32, (w, h))  # resize to integer multiple of 32
    image = image.resize((512, 512), resample=PIL.Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.*image - 1.

In [11]:
config="configs/stable-diffusion/v1-inference.yaml"
ckpt="sd-v1-4.ckpt"
config = OmegaConf.load(f"{config}")
model = load_model_from_config(config, f"{ckpt}")
sampler = DDIMSampler(model)

Loading model from sd-v1-4.ckpt
Global Step: 470000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [12]:
# !python main.py --base configs/stable-diffusion/v1-finetune.yaml -t --actual_resume sd-v1-4.ckpt -n run101  --gpus 0,  --data_root data/vangogh

In [13]:
torch.cuda.current_device()


0

In [42]:
def main(prompt = '', content_dir = '', style_dir='',ddim_steps = 50,strength = 0.5, model = None, seed=42):
    ddim_eta=0.0
    n_iter=1
    C=4
    f=8
    n_samples=1
    n_rows=0
    scale=10.0

    precision="autocast"
    outdir="../EvaluationImages/Results/Diffusion/andre"
    seed_everything(seed)
    input_img = cv2.imread(content_dir)
    x,y = input_img.shape[1],input_img.shape[0]

    os.makedirs(outdir, exist_ok=True)
    outpath = outdir

    batch_size = n_samples
    n_rows = n_rows if n_rows > 0 else batch_size
    data = [batch_size * [prompt]]


    sample_path = os.path.join(outpath, "samples")
    os.makedirs(sample_path, exist_ok=True)
    base_count = len(os.listdir(sample_path))
    grid_count = len(os.listdir(outpath)) + 10

    style_image = load_img(style_dir).to(device)
    style_image = repeat(style_image, '1 ... -> b ...', b=batch_size)
    style_latent = model.get_first_stage_encoding(model.encode_first_stage(style_image))  # move to latent space

    content_name =  content_dir.split('/')[-1].split('.')[0]
    content_image = load_img(content_dir).to(device)
    content_image = repeat(content_image, '1 ... -> b ...', b=batch_size)
    content_latent = model.get_first_stage_encoding(model.encode_first_stage(content_image))  # move to latent space

    init_latent = content_latent

    sampler.make_schedule(ddim_num_steps=ddim_steps, ddim_eta=ddim_eta, verbose=False)

    assert 0. <= strength <= 1., 'can only work with strength in [0.0, 1.0]'
    t_enc = int(strength * ddim_steps)
    print(f"target t_enc is {t_enc} steps")

    precision_scope = autocast if precision == "autocast" else nullcontext
    with torch.no_grad():
        with precision_scope("cuda"):
            with model.ema_scope():
                tic = time.time()
                all_samples = list()
                for n in trange(n_iter, desc="Sampling"):
                    for prompts in tqdm(data, desc="data"):
                        uc = None
                        if scale != 1.0:
                            uc = model.get_learned_conditioning(batch_size * [""], style_image)
                        if isinstance(prompts, tuple):
                            prompts = list(prompts)

                        c= model.get_learned_conditioning(prompts, style_image)

                        # img2img

                        # stochastic encode
                        # z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc]*batch_size).to(device))

                        # stochastic inversion
                        t_enc = int(strength * 1000)
                        x_noisy = model.q_sample(x_start=init_latent, t=torch.tensor([t_enc]*batch_size).to(device))
                        model_output = model.apply_model(x_noisy, torch.tensor([t_enc]*batch_size).to(device), c)
                        z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc]*batch_size).to(device),\
                                                          noise = model_output, use_original_steps = True)

                        t_enc = int(strength * ddim_steps)
                        samples = sampler.decode(z_enc, c, t_enc,
                                                unconditional_guidance_scale=scale,
                                                 unconditional_conditioning=uc,)
                        print(z_enc.shape, uc.shape, t_enc)

                        # txt2img
            #             noise  =torch.randn_like(content_latent)
            #             samples, intermediates =sampler.sample(ddim_steps,1,(4,512,512),c,verbose=False, eta=1.,x_T = noise,
            #    unconditional_guidance_scale=scale,
            #    unconditional_conditioning=uc,)

                        x_samples = model.decode_first_stage(samples)

                        x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)

                        for x_sample in x_samples:
                            x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                            base_count += 1
                        all_samples.append(x_samples)

                # additionally, save as grid
                grid = torch.stack(all_samples, 0)
                grid = rearrange(grid, 'n b c h w -> (n b) c h w')
                grid = make_grid(grid, nrow=n_rows)

                # to image
                grid = 255. * rearrange(grid, 'c h w -> h w c').cpu().numpy()
                output = Image.fromarray(grid.astype(np.uint8))
                output.save(os.path.join(outpath, content_name+'-'+'modern.png'))
                # Image.fromarray(grid.astype(np.uint8)).save(os.path.join(outpath, f'grid-{grid_count:04}.png'))
                grid_count += 1

                toc = time.time()
    print(type(output), output.size, content_image.shape[2], content_image.shape[3],x,y)
    output = output.resize((x, y))
    return output

In [15]:
# !python main.py --base configs/stable-diffusion/v1-finetune.yaml --train --actual_resume sd-v1-4.ckpt -n mm_train1 --gpus 2, --data_root ../MM_Generations/mmgeneration/data/monet2photo/monet2photo/trainA

In [19]:
import cv2

In [41]:
model.embedding_manager.load('andre-derain_embeddings.pt')
model = model.to(device)

find keys: dict_keys(['string_to_token', 'attention'])


In [17]:
ls

andre-derain_embeddings.pt  evaluation/       logs/                   README.md
configs/                    evaluator.ipynb   longhair_embeddings.pt  scripts/
contour_point_simple.jpg    Images/           main.py                 sd-v1-4.ckpt
contours_none_image1.jpg    image_thres1.jpg  models/                 setup.py
contours_simple_image1.jpg  InST.ipynb        modern_embeddings.pt    woman_embeddings.pt
data/                       ldm/              outputs/
environment.yaml            LICENSE           __pycache__/


In [43]:
content = '../EvaluationImages/Content/1_content.jpg'
# style = '../MM_Generations/mmgeneration/data/monet2photo/monet2photo/trainA/01327.jpg'
input_img = cv2.imread(content)
# style_img = cv2.imread(style)

# model.embedding_manager.load('modern_embeddings.pt')
# model = model.to(device)
generated1 = main(prompt = '*', \
     content_dir = content, \
     style_dir = style, \
     ddim_steps = 60, \
     strength = 0.4, \
     seed=42, \
     model = model)

# model.embedding_manager.load('andre-derain_embeddings.pt')
# model = model.to(device)
# style = '../MM_Generations/mmgeneration/data/monet2photo/monet2photo/trainA/01322.jpg'
# generated2 = main(prompt = '*', \
#      content_dir = content, \
#      style_dir = style, \
#      ddim_steps = 60, \
#      strength = 0.45, \
#      seed=42, \
#      model = model)

from matplotlib import pyplot as plt
plt.figure(figsize=(20, 20))
plt.subplot(1,2,1)
plt.title('content image')
input_RGB = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)
plt.xticks([])
plt.yticks([])
plt.imshow(input_RGB)

# plt.subplot(1,3,2)
# plt.title('style')
# style_RGB = cv2.cvtColor(style_img, cv2.COLOR_BGR2RGB)
# plt.xticks([])
# plt.yticks([])
# plt.imshow(style_RGB)

plt.subplot(1,2,2)
plt.title('generated in modern style')
plt.xticks([])
plt.yticks([])
plt.imshow(generated1)

# plt.subplot(1,2,2)
# plt.title('generated')
# plt.xticks([])
# plt.yticks([])
# plt.imshow(generated2)
# plt.show()